In [1]:
from datetime import datetime as dt, timedelta as td
import requests

In [2]:
url = "https://query1.finance.yahoo.com/v7/finance/options/BAC"
#?formatted=true&crumb=bCkXlpLUQ.w&lang=en-US&region=US&date=1484265600&corsDomain=finance.yahoo.com

In [3]:
page = requests.get(url)
response = page.json()
if response['optionChain']['error']:
    print(response['optionChain']['error'])

results = response['optionChain']['result']
results

[{'expirationDates': [1485475200,
   1486080000,
   1486684800,
   1487289600,
   1487894400,
   1488499200,
   1489708800,
   1492732800,
   1495152000,
   1497571200,
   1503014400,
   1513296000,
   1516320000,
   1547769600],
  'hasMiniOptions': False,
  'options': [{'calls': [{'ask': 7.25,
      'bid': 7.05,
      'change': 0.39999962,
      'contractSize': 'REGULAR',
      'contractSymbol': 'BAC170127C00015500',
      'currency': 'USD',
      'expiration': 1485475200,
      'impliedVolatility': 1.99609376953125,
      'inTheMoney': True,
      'lastPrice': 7.2,
      'lastTradeDate': 1484929103,
      'openInterest': 7,
      'percentChange': 5.882347,
      'strike': 15.5,
      'volume': 8},
     {'ask': 7.8,
      'bid': 6.85,
      'change': 0.5999999,
      'contractSize': 'REGULAR',
      'contractSymbol': 'BAC170127C00016000',
      'currency': 'USD',
      'expiration': 1485475200,
      'impliedVolatility': 3.3515641210937495,
      'inTheMoney': True,
      'lastPrice':

In [4]:
result = results[0]
dates = [dt.fromtimestamp(tm) for tm in result['expirationDates']]
# we can use these dates to get data for a specific expiration date
# an example url: https://query1.finance.yahoo.com/v7/finance/options/BAC?date=1484265600 
# the first date is default
dates

[datetime.datetime(2017, 1, 27, 2, 0),
 datetime.datetime(2017, 2, 3, 2, 0),
 datetime.datetime(2017, 2, 10, 2, 0),
 datetime.datetime(2017, 2, 17, 2, 0),
 datetime.datetime(2017, 2, 24, 2, 0),
 datetime.datetime(2017, 3, 3, 2, 0),
 datetime.datetime(2017, 3, 17, 2, 0),
 datetime.datetime(2017, 4, 21, 3, 0),
 datetime.datetime(2017, 5, 19, 3, 0),
 datetime.datetime(2017, 6, 16, 3, 0),
 datetime.datetime(2017, 8, 18, 3, 0),
 datetime.datetime(2017, 12, 15, 2, 0),
 datetime.datetime(2018, 1, 19, 2, 0),
 datetime.datetime(2019, 1, 18, 2, 0)]

In [5]:
option = result['options'][0]
calls = option['calls']
calls[:2]

[{'ask': 7.25,
  'bid': 7.05,
  'change': 0.39999962,
  'contractSize': 'REGULAR',
  'contractSymbol': 'BAC170127C00015500',
  'currency': 'USD',
  'expiration': 1485475200,
  'impliedVolatility': 1.99609376953125,
  'inTheMoney': True,
  'lastPrice': 7.2,
  'lastTradeDate': 1484929103,
  'openInterest': 7,
  'percentChange': 5.882347,
  'strike': 15.5,
  'volume': 8},
 {'ask': 7.8,
  'bid': 6.85,
  'change': 0.5999999,
  'contractSize': 'REGULAR',
  'contractSymbol': 'BAC170127C00016000',
  'currency': 'USD',
  'expiration': 1485475200,
  'impliedVolatility': 3.3515641210937495,
  'inTheMoney': True,
  'lastPrice': 7.1,
  'lastTradeDate': 1484321104,
  'openInterest': 4,
  'percentChange': 9.230768,
  'strike': 16.0,
  'volume': 43}]

In [6]:
puts = option['puts']
puts[-2:]

[{'ask': 5.55,
  'bid': 5.45,
  'change': 0.0,
  'contractSize': 'REGULAR',
  'contractSymbol': 'BAC170127P00028500',
  'currency': 'USD',
  'expiration': 1485475200,
  'impliedVolatility': 1.0000000000000003e-05,
  'inTheMoney': True,
  'lastPrice': 5.35,
  'lastTradeDate': 1484369513,
  'openInterest': 0,
  'percentChange': 0.0,
  'strike': 28.5,
  'volume': 200},
 {'ask': 6.05,
  'bid': 5.95,
  'change': 0.0,
  'contractSize': 'REGULAR',
  'contractSymbol': 'BAC170127P00029000',
  'currency': 'USD',
  'expiration': 1485475200,
  'impliedVolatility': 1.0000000000000003e-05,
  'inTheMoney': True,
  'lastPrice': 5.85,
  'lastTradeDate': 1484369513,
  'openInterest': 0,
  'percentChange': 0.0,
  'strike': 29.0,
  'volume': 100}]

# Implementation

In [7]:
# There is a lot of data to load every minute, so we have to do this in parrallel
# 'threading' module fits well for this purpose
# I'm going to use two queues to connect our main programm and thread workers: 
# the first is for tasks, the other - for results
# You can find some examples of using 'queue'-module here
# https://docs.python.org/3/library/queue.html#queue.Queue.join
import queue
import requests
import threading


def load_data(time):
    
    num_worker_threads = 1000
    
    def worker():
        while True:
            args = q.get()
            if args is None:
                break
            get_transactions(q, r, *args)  # <-- see definition of this function below
            q.task_done()

    q = queue.Queue() # queue with arguments for the 'get_transactions' function
    r = queue.Queue() # queue with raw responses

    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker)
        t.start()
        threads.append(t)

    for stock in STOCKS:
        q.put((stock, None))

    # block until all tasks are done
    q.join()

    # stop workers
    for i in range(num_worker_threads):
        q.put(None)

    for t in threads:
        t.join()
        
    save_rows(r, time)
    
    
def get_transactions(queue, resp_queue, stock, date=None):     
    base_url = "https://query1.finance.yahoo.com/v7/finance/options/{}".format(stock)
    if date is not None:
        url = "{}?date={}".format(base_url, date)
    else:
        url = base_url
    try:
        page = requests.get(url)
    except Exception as e:
        print(e)
    else:
        response = page.json()
        if response['optionChain']['error']: # if any error
            print(response['optionChain']['error'])

        else:
            results = response['optionChain']['result']
            for result in results:
                price = result['quote']['regularMarketPrice']
                for option in result['options']:
                    resp_queue.put(
                        (stock, price, option['calls'], option['puts'])
                    )

                if date is None:
                    for tm in result['expirationDates'][1:]:
                        queue.put((stock, tm))

In [8]:
# Functions which take response from the response queue and put to the 'largest_transactions' list
def save_rows(q, time):
    while True:
        try:
            stock, price, calls, puts = q.get(block=False)
        except queue.Empty:
            break
        else:
            for call in calls:
                save_row(stock, price, 'Calls', time, call)
            for put in puts:
                save_row(stock, price, 'Puts', time, put)

def save_row(s, price, t, time, raw):
    contact = raw['contractSymbol']    
    previous_volume = transaction_volumes.get(contact)
    volume = raw['volume']
    transaction_volumes[contact] = volume
    
    if previous_volume is not None:        
        volume_diff = volume - previous_volume
        if volume_diff > 0:
            last_price = raw['lastPrice']
            strike = raw['strike']
            ask = raw['ask']
            bid = raw['bid']
            largest_transactions.append(
                dict(
                    stock=s,
                    price=price,
                    type=t,
                    time=time,
                    expiration=dt.fromtimestamp(raw['expiration']),
                    strike=strike,
                    contact=contact,
                    last_price=last_price,
                    bid=bid,
                    ask=ask,
                    volume=volume,
                    open_interest=raw['openInterest'],
                    implied_volatility=raw['impliedVolatility'],
                    volume_diff=volume_diff,
                    cost=volume_diff * 100 * last_price,
                    strike_div_price=(strike / price - 1) * 100,
                    last_price_position="{:.0f}".format((1 - (ask-last_price) / (ask-bid)) * 100) if ask != bid else None
                )
            )

In [9]:
# This function will display the table
from IPython.display import clear_output, display, HTML

headers = (
    "Stock", "Share price", "Options", "Volume", "Strike", "Strike /Share price", 
    "Expiration", "Last price", "Bid", "Ask", "Last price position",  "Value", "Time of trade",
)
row = """
<td>{stock}</td><td>{price}</td><td>{type}</td><td>{volume_diff}</td>
<td>{strike}</td><td>{strike_div_price:.2f}%</td><td>{expiration:%b %d, %Y}</td>
<td>{last_price}</td><td>{bid}</td><td>{ask}</td><td>{last_price_position}%</td>
<td>${cost:,.0f}</td><td>{time:%I:%M%p}</td>
"""

def display_data():    
    table_rows = (row.format(**row_data) for row_data in largest_transactions)           
    clear_output()
    display(
        HTML(
            '<table><tr><th>{}</th></tr><tr>{}</tr></table>'.format(
                "</th><th>".join(headers),
                "</tr><tr>".join(table_rows)
            )
        )
    )   

In [ ]:
from datetime import datetime as dt, timedelta as td
from pytz import timezone
from time import sleep
import heapq

tz = timezone('America/Los_Angeles')  # timezone produces time with PST or PDT tzname depending on the date
STOCKS = ['AAPL', 'BAC', 'NVDA', 'AMZN', 'FB']

last_time = None
count = 1
largest_transactions = []  # here we will store top entries
transaction_volumes = {}  # latest volumes for transactions

while True:    
    now = dt.now(tz=tz)
    start = now.replace(hour=6, minute=30, second=0, microsecond=0)
    stop = start.replace(hour=13, minute=0)
    
    if start <= now <= stop: # proper time for getting data
        
        if last_time: # no more often than once per minute
            diff = 60 - (now - last_time).seconds            
            if diff > 0:
                sleep(diff)  
        
        # the main functionality
        load_data(now)
        # it's efficient way to keep only top 20 entries in the 'largest_transactions' list
        largest_transactions = heapq.nlargest(20, largest_transactions, key=lambda i: i['cost'])
        display_data()
        
        # some statistics
        print("last update", now)
        print("prev update", last_time) 
        print("update time", dt.now(tz=tz) - now)
        print("iterations", count)        
        count += 1        
        last_time = now        
    else:
        largest_transactions = [] # drop transactions
        if now < start:
            diff = start - now
            print("time to start:", diff)
            
        else:
            start_tommorow = start + td(days=1)
            print("start tommorow at", start_tommorow)
            diff = start_tommorow - now
        
        sleep(diff.seconds + diff.microseconds * 0.1 ** 6)
    

Stock,Share price,Options,Volume,Strike,Strike /Share price,Expiration,Last price,Bid,Ask,Last price position,Value,Time of trade
AMZN,813.9019,Calls,42,830.0,1.98%,"Feb 17, 2017",19.65,19.45,19.75,67%,"$82,530",10:58AM
AMZN,813.9019,Puts,143,740.0,-9.08%,"Feb 17, 2017",4.5,4.5,4.7,0%,"$64,350",10:58AM
AMZN,813.9019,Calls,1,450.0,-44.71%,"Jun 16, 2017",313.4,294.05,297.1,634%,"$31,340",10:59AM
AAPL,120.07,Puts,393,120.0,-0.06%,"Jan 27, 2017",0.74,0.73,0.74,100%,"$29,082",10:58AM
AMZN,813.9019,Calls,28,810.0,-0.48%,"Jan 27, 2017",9.13,9.3,9.55,-68%,"$25,564",10:58AM
AMZN,813.9019,Calls,54,820.0,0.75%,"Jan 27, 2017",4.6,4.5,4.75,40%,"$24,840",10:58AM
AMZN,813.9019,Calls,5,780.0,-4.17%,"Jan 27, 2017",34.7,33.55,34.65,105%,"$17,350",10:58AM
AMZN,813.9019,Puts,7,812.5,-0.17%,"Feb 03, 2017",23.52,23.25,23.55,90%,"$16,464",10:58AM
AMZN,813.9019,Calls,19,812.5,-0.17%,"Jan 27, 2017",7.94,7.85,8.05,45%,"$15,086",10:58AM
AMZN,813.9019,Puts,55,800.0,-1.71%,"Jan 27, 2017",2.34,2.25,2.39,64%,"$12,870",10:58AM


last update 2017-01-23 10:59:52.002576-08:00
prev update 2017-01-23 10:58:44.049948-08:00
update time 0:00:08.116877
iterations 3
